In [1]:
from os import chdir
chdir('../')
%matplotlib inline
from lib import *

from time import time

pd.options.display.precision = 4
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Lato'
mpl.rcParams['font.weight'] = 700
sns.set(font='Lato', font_scale=1)
sns.set()
np.random.seed(42)
suppress_warnings()

In [2]:
feature = pd.read_pickle('/home/jovyan/work/DSI/projects/project_3/data/feature')
target= pd.read_pickle('/home/jovyan/work/DSI/projects/project_3/data/target')

In [3]:
display(feature.shape)
display(target.shape)

(10768, 1000)

(10768, 1)

In [4]:
feature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10768 entries, 519 to 219787
Columns: 1000 entries, feat_0002 to feat_4996
dtypes: float64(1000)
memory usage: 82.2 MB


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_classif
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso, LogisticRegression,LinearRegression

# Step1 : Benchmarking
#### KNC

In [6]:
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

In [7]:
benchmarking_pipe_for_gs = Pipeline([
    ('knc',KNeighborsClassifier())
                        ])

In [8]:
params = {}

In [9]:
gs_pipe = gspipe = GridSearchCV(benchmarking_pipe_for_gs,
                      param_grid=params,
                      n_jobs=-1,
                      cv=ShuffleSplit(random_state=42)
                     )

In [10]:
gspipe.fit(feature[:220], np.ravel(target[:220].values))

GridSearchCV(cv=ShuffleSplit(n_splits=10, random_state=42, test_size=0.1, train_size=None),
       error_score='raise',
       estimator=Pipeline(steps=[('knc', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))]),
       fit_params={}, iid=True, n_jobs=-1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [11]:
gspipe.best_score_

0.52272727272727271

In [12]:
gspipe_results = pd.DataFrame(gspipe.cv_results_)
gspipe_results.sort_values('rank_test_score', ascending=True).T

,0
mean_fit_time,0.003678
mean_score_time,0.007675
mean_test_score,0.5227
mean_train_score,0.7227
params,{}
rank_test_score,1
split0_test_score,0.6818
split0_train_score,0.6717
split1_test_score,0.4091
split1_train_score,0.7222


In [13]:
gspipe_results['mean_fit_time'].sum()

0.0036775588989257811

In [14]:
gspipe_results.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 30 columns):
mean_fit_time         1 non-null float64
mean_score_time       1 non-null float64
mean_test_score       1 non-null float64
mean_train_score      1 non-null float64
params                1 non-null object
rank_test_score       1 non-null int32
split0_test_score     1 non-null float64
split0_train_score    1 non-null float64
split1_test_score     1 non-null float64
split1_train_score    1 non-null float64
split2_test_score     1 non-null float64
split2_train_score    1 non-null float64
split3_test_score     1 non-null float64
split3_train_score    1 non-null float64
split4_test_score     1 non-null float64
split4_train_score    1 non-null float64
split5_test_score     1 non-null float64
split5_train_score    1 non-null float64
split6_test_score     1 non-null float64
split6_train_score    1 non-null float64
split7_test_score     1 non-null float64
split7_train_score    1 non-null float64


In [15]:
print("Best Score .1% data:",gspipe.best_score_)
    
### Bench Marking for .1% (220 rows)
picked_GS_BenchMarkig_220row = \
        pd.DataFrame.to_pickle(gspipe_results,\
        '/home/jovyan/work/DSI/projects/project_3/data/GS_BenchMarkig_knc_pt1_percent')

Best Score .1% data: 0.522727272727


In [16]:
### Bench Marking for .5% (1100 rows)

gspipe.fit(feature[:1100], np.ravel(target[:1100].values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time .5% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score .5% data:",gspipe.best_score_)
gspipe_results_1100 = pd.DataFrame(gspipe.cv_results_)
picked_GS_BenchMarkig1100row = \
        pd.DataFrame.to_pickle(gspipe_results_1100,\
        '/home/jovyan/work/DSI/projects/project_3/data/GS_BenchMarkig_knc_pt5_percent')
    

Fit time .5% data:  0.0229219198227
Best Score .5% data: 0.541818181818


In [17]:
### Bench Marking for 1% (2200 rows)

gspipe.fit(feature[:2200], np.ravel(target[:2200].values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time 1% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score 1% data:",gspipe.best_score_)
gspipe_results_2200 = pd.DataFrame(gspipe.cv_results_)
picked_GS_BenchMarkig_2200row = \
        pd.DataFrame.to_pickle(gspipe_results_2200,\
        '/home/jovyan/work/DSI/projects/project_3/data/GS_BenchMarkig_knc_1_percent')
    

Fit time 1% data:  0.0536406040192
Best Score 1% data: 0.534545454545


In [18]:
### Bench Marking for 5% (11000 rows)

gspipe.fit(feature, np.ravel(target.values))
gspipe_results = pd.DataFrame(gspipe.cv_results_)
print("Fit time 5% data: ", gspipe_results['mean_fit_time'].sum())
print("Best Score 5% data:",gspipe.best_score_)
gspipe_results_11000 = pd.DataFrame(gspipe.cv_results_)
picked_GS_BenchMarkig_11000row = \
        pd.DataFrame.to_pickle(gspipe_results_11000,\
        '/home/jovyan/work/DSI/projects/project_3/data/GS_BenchMarkig_knc_5_percent')

Fit time 5% data:  0.361035656929
Best Score 5% data: 0.523398328691
